In [113]:
import os
import re
import pandas as pd

# Find the file name(s) based on a given protein name

In [288]:

proteinname = 'Af_PCC|1643'
regex_pattern = r">" + re.escape(proteinname) + r" \["
match_reuslt = re.match(regex_pattern, text)


def get_filenames_from_proteinname(proteinname: str):
    """
    The input is a string indicting the name of specific protein.
    This function is going to find the file names that contain this protein name in the text.
    """
    def exist_deisred_name(file_name, proteinname):
        with open(os.path.join(directory, file_name), 'r') as f:
            content = '\n'.join(f.readlines())
            
            proteinname = proteinname.replace('|', '\|')
            regex_pattern = re.compile(f">{proteinname} \[")
            match_reuslt = re.search(regex_pattern, content)
            
            if match_reuslt is None:
                return False
            else:
                return True

    directory = 'D:/Pycharm/project/orthologues_run_Friday'
    file_names = os.listdir(directory)
    desired_names = [x for x in file_names if exist_deisred_name(x, proteinname)]
    return desired_names

In [314]:
def find_species_gene_from_uml_rec():
    """
    Find the species_gene from each .uml_rec files
    """
    
    def find_species_genes(file_name):
        """
        Find the species_gene from a given file 
        """
        with open(file_name, 'r') as f:
            content = f.readlines()
        result = content[11].split(':')[0].replace('(', "").split('.')[0]
        return result
    
    fold_name = r'D:\OneDrive - University of Calgary\bioinformatics\homework\ancestral reconstruction\root2'
    fruit = []
    target_file_names = [x for x in os.listdir(fold_name) if x.endswith('.uml_rec')]
    for i, uml_rec in enumerate(target_file_names):
        file_path = os.path.join(fold_name, uml_rec)
        fruit.append([uml_rec, find_species_genes(file_path)])
        
        # print the progress
        if i % 1000 == 0:
            print(f"Processing progess: {i}/{len(target_file_names)}")
    
    df = pd.DataFrame(fruit, columns=['uml_rec', 'species_gene'])
    return df


species_gene_df = find_species_gene_from_uml_rec()
species_gene_df.to_csv(r"D:\OneDrive - University of Calgary\bioinformatics\homework\ancestral reconstruction\fruit.csv", index=False)
print(species_gene_df.head(20))

Processing progess: 0/4264
Processing progess: 1000/4264
Processing progess: 2000/4264
Processing progess: 3000/4264
Processing progess: 4000/4264
                                        uml_rec    species_gene
0       re_root.newick_0.aln.ufboot.ale.uml_rec     G_SBC|24918
1       re_root.newick_1.aln.ufboot.ale.uml_rec  P_yuhuli|54541
2   re_root.newick_1.faa.aln.ufboot.ale.uml_rec  P_yuhuli|54541
3      re_root.newick_10.aln.ufboot.ale.uml_rec      Af_PCC|972
4     re_root.newick_100.aln.ufboot.ale.uml_rec      Af_PCC|845
5    re_root.newick_1000.aln.ufboot.ale.uml_rec  P_yuhuli|53527
6    re_root.newick_1001.aln.ufboot.ale.uml_rec     G_BBD|14948
7    re_root.newick_1002.aln.ufboot.ale.uml_rec    G_9228|12080
8    re_root.newick_1005.aln.ufboot.ale.uml_rec     G_BBD|18402
9    re_root.newick_1006.aln.ufboot.ale.uml_rec      Af_PCC|835
10   re_root.newick_1007.aln.ufboot.ale.uml_rec     G_7105|9012
11   re_root.newick_1008.aln.ufboot.ale.uml_rec    G_9228|12584
12   re_root.newick_1

# Concat all csv file names together

In [318]:
fold_name = r"D:\OneDrive - University of Calgary\bioinformatics\homework\ancestral reconstruction\gene family at each node"

df_2 = pd.DataFrame()
for file_name in os.listdir(fold_name):
    df_2 = pd.concat([df_2, pd.read_csv(os.path.join(fold_name, file_name))])

df_merge = pd.merge(df_2, species_gene_df.rename(columns={"uml_rec": "Gene_family"}), on='Gene_family', how='left')
df_merge.to_csv(r"D:\OneDrive - University of Calgary\bioinformatics\homework\ancestral reconstruction\df_merge.csv", index=False)


In [118]:
df_merge = pd.read_csv(r"D:\OneDrive - University of Calgary\bioinformatics\homework\ancestral reconstruction\df_merge.csv")

In [333]:
def extract_protein(file_name: str) -> str:
    """
    Extract the protein from the first line.
    The regex rule is:
    Start from: a period with several digits and a space
    End before: a space and '(Bacteria'
    """
    with open(os.path.join(directory, file_name), 'r') as f:
        content = f.readline()
    search_result = re.search(r'\.\d+ (.+) \(Bacteria', content)
    if search_result is not None:
        return search_result.group(1)
    else:
        return None

In [336]:

file_name_ = []
file_protein_ = []
for i, proteinname in enumerate(df_merge['species_gene']):
    file_names_list_for_proteinname = get_filenames_from_proteinname(proteinname)
    file_name = file_names_list_for_proteinname[0]
    protein = extract_protein(file_name)
    file_name_.append(file_name)
    file_protein_.append(protein)
    
    if i % 100 == 0:
        print(f"{i}/{len(df_merge)} \t Finshed.")

df_merge['gene_cluster'] = file_name_
df_merge['protein'] = file_protein_
df_merge.to_csv(r'C:\Users\ylc_c\OneDrive\桌面\df_merge.csv', index=False)

0/1438 	 Finshed.
100/1438 	 Finshed.
200/1438 	 Finshed.
300/1438 	 Finshed.
400/1438 	 Finshed.
500/1438 	 Finshed.
600/1438 	 Finshed.
700/1438 	 Finshed.
800/1438 	 Finshed.
900/1438 	 Finshed.
1000/1438 	 Finshed.
1100/1438 	 Finshed.
1200/1438 	 Finshed.
1300/1438 	 Finshed.
1400/1438 	 Finshed.
